In [305]:
from typing import  List

from magicgui.widgets import PushButton, FloatSlider, ComboBox, Label
from magicgui import magicgui
from skimage import data
import napari


@magicgui(
    auto_call = True,
    new_pair_button = {"widget_type": "PushButton", 'label':'new pair'},
    test = {"widget_type": "FloatSlider", 'max': 2},
    images = {'visible':False}
)
def napari_plugin(
    new_pair_button,
    images: napari.layers.Image,
    test:float = 1.0,
) -> List[napari.types.LayerDataTuple]:

    mixing_data = napari_plugin.asdict()
    pairs = [p.split(' ')[-1] for p in list(mixing_data) if 'alpha' in p]
    print(pairs)
    
    unmixed = []
    for p in pairs:
        source = mixing_data['source_'+p]
        sink = mixing_data['sink_'+p]
        alpha = mixing_data['alpha_'+p]
    
        if source is not None and sink is not None:
            sink_ = sink.data - alpha*source.data
            #unmixed.append((sink_, {'name':'pair '+p}, 'image'))
            
    if len(unmixed) > 0:
        return unmixed


# # # set up callbacks
@napari_plugin.new_pair_button.clicked.connect
def new_pair():
    
    # Get unique index for image pairs
    widget_names = list(napari_plugin.asdict())
    print(widget_names)
    index = (len(widget_names)-2)
#     while 'delete_'+str(index) in widget_names:
#         index += 1
    index = str(index)
    print(index)
    im_choices = (None,) + napari_plugin.images.choices
          
    # define widgets with names
    label = Label(label = 'pair '+index, name='pair_'+index)
    delete_button = PushButton(label='delete', name='delete_'+index) 
    source = ComboBox(choices=im_choices, label='source', name='source_'+index)
    sink = ComboBox(choices=im_choices, label='sink', name='sink_'+index)
    alpha = FloatSlider(max=2.0, label='alpha', value = 0.0, name='alpha_'+index)
    
#     # define widget without names
#     label = Label(label='pair '+index) 
#     source = ComboBox(choices=im_choices, label='source')
#     sink = ComboBox(choices=im_choices, label='sink')
#     alpha = FloatSlider(max=2.0, label='alpha', value = 0.0)
    
    # delete button callback
    @delete_button.changed.connect
    def delete_pair():
        index = delete_button.name.split('_')[-1]
        for w in ['pair', 'delete','source', 'sink', 'alpha']:
            napari_plugin.remove(w+'_'+index)
    
    #Add widgets
    new_pair = [label, delete_button, source, sink, alpha]
    #new_pair =  [label, source, sink, alpha]
    napari_plugin.extend(new_pair)
    
    widget_names = list(napari_plugin.asdict())
    print(widget_names)
    


rgb_im = data.astronaut()
viewer = napari.Viewer()
viewer.add_image(rgb_im[:,:,0].squeeze(), name= 'red', blending='additive', colormap='red')
viewer.add_image(rgb_im[:,:,1].squeeze(), name= 'green', blending='additive', colormap='green')
viewer.add_image(rgb_im[:,:,2].squeeze(), name= 'blue', blending='additive', colormap='blue')
widget = viewer.window.add_dock_widget(napari_plugin, name='PICASSO')

test


In [159]:
from magicgui.widgets import PushButton, FloatSlider, Container, ComboBox
from napari.types import ImageData

# make some widgets
add_new_button = PushButton(name='add')

# set up callbacks
@add_new_button.changed.connect
def add_pair():
    container.extend([Container(widgets=[ComboBox(choices=[ImageData]),
                                         ComboBox(choices=[ImageData]),
                                         FloatSlider(max=2),
                                        ])])

# create a container to hold the widgets:
container = Container(name='BigContainer', widgets=[add_new_button])
rgb_im = data.astronaut()
viewer = napari.Viewer()
viewer.add_image(rgb_im[:,:,0].squeeze(), name= 'red')
viewer.add_image(rgb_im[:,:,1].squeeze(), name= 'green')
viewer.window.add_dock_widget(container)

In [160]:
from magicgui.widgets import FunctionGui

def my_function(source, sink, alpha):
    return sink - alpha*source

class MyGui(FunctionGui):
    def __init__(self):
        super().__init__(
          my_function,
          call_button=True,
          layout='vertical',
          param_options={...}
        )
        # do whatever other initialization you want here